In [126]:
import json
import codecs
import tensorflow as tf
import collections

In [12]:
with open('/crimea/geeticka/data/relation_extraction/semeval2010/pre-processed/original/bert/train_original_border_50.json') as file:
    for line in file.readlines():
        data = json.loads(line)
        print(data)
        break

{'linex_index': 0, 'features': [{'token': '[CLS]', 'layers': [{'index': -1, 'values': [-0.615448, 0.176578, -0.31343, 0.218013, 0.233812, 0.446207, 0.032351, -0.111474, 0.635218, -0.131597, 0.22472, -0.063258, 0.316621, -0.649463, 0.132566, 0.673724, -0.448832, -0.361726, -0.241304, 0.218446, 0.092953, 0.319181, 0.155885, 0.147426, -0.145467, 0.055356, 0.148498, -0.101869, 0.175572, 0.38413, -0.339514, 0.760027, -0.601023, -0.286325, 0.168428, 0.222484, -0.251318, -0.301582, -0.055841, -0.333101, -0.42574, 0.100072, -0.432756, -0.332847, 0.211059, 0.439489, 0.256271, 0.069989, 0.176118, 0.154655, 0.133832, 0.366228, -0.546357, 0.172038, 0.402881, 0.935933, 0.07003, -0.239716, -0.211174, -0.060708, 1.28376, -0.409187, -0.185515, 0.174601, -0.372036, -0.144544, 0.571965, 0.333008, 0.205351, -0.010121, -0.103814, -0.062724, 0.390606, 0.640407, -0.245448, 0.187674, -0.275909, -0.919277, -0.342203, -0.315296, -0.288465, -0.453292, -0.278862, 0.109965, -0.10736, -0.21802, -0.046072, 0.119483

In [43]:
# because the dumping of the embeddings was done using pytorch, refer to their method of writing to 
# figure out how to do the reading
# https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/examples/extract_features.py
# refer to the above to figure out how to read this file
# need to import collections
def write_bert_tokens(input_filename, output_filename):
    with open(input_filename, 'r', encoding='utf-8') as input_file, open(output_filename, 'w', encoding='utf-8' as output_file:
        for line in input_file.readlines():
            if data['features'][0]['token'] != '[CLS]': raise Exception("The first token has to be CLS!")
            if data['features'][-1]['token'] != '[SEP]': raise Exception("The last token has to be SEP!")
            output_json = collections.OrderedDict()
            data = json.loads(line)
            output_json['linex_index'] = data['linex_index']
            features = data['features'] # basically for all features['token'] that starts with ##, add up the values
             # for the respective indexes to put the words back together, ignore [CLS] and [SEP] tokens
            new_feature_map = generate_feature_map_without_word_piece(features) # this new feature map needs to be 
            # called layers because things have now been shuffled. 
            output_json['layers'] = new_feature_map
            output_filename.write(json.dumps(output_json) + "\n")                                                            

In [49]:
a = [1]
a.extend([2,3,4])

In [127]:
features = [{'token': '3', 'layers': [{'index': -1, 'values': [-1, -3]},
                                    {'index': -2, 'values': [-4, -2]},
                                    {'index': -3, 'values': [-2, -1]},
                                    {'index': -4, 'values': [-1,2]}]},
            {'token': 'a', 'layers': [{'index': -1, 'values': [1,2]},
                                     {'index': -2, 'values': [3,4]},
                                     {'index': -3, 'values': [4,5]},
                                     {'index': -4, 'values': [2,1]}]},
           {'token': '##b', 'layers': [{'index': -1, 'values': [-1, -3]},
                                      {'index': -2, 'values': [-2, -1]},
                                      {'index': -3, 'values': [-3, -2]},
                                      {'index': -4, 'values': [-1, -1]}]},
           {'token': '3', 'layers': [{'index': -1, 'values': [-1, -3]},
                                    {'index': -2, 'values': [-4, -2]},
                                    {'index': -3, 'values': [-2, -1]},
                                    {'index': -4, 'values': [-1,2]}]},
           {'token': '##b', 'layers': [{'index': -1, 'values': [-1, -3]},
                                      {'index': -2, 'values': [-2, -1]},
                                      {'index': -3, 'values': [-3, -2]},
                                      {'index': -4, 'values': [-1, -1]}]},
           {'token': '##b', 'layers': [{'index': -1, 'values': [-1, -3]},
                                      {'index': -2, 'values': [-2, -1]},
                                      {'index': -3, 'values': [-3, -2]},
                                      {'index': -4, 'values': [-1, -1]}]},
           {'token': '3', 'layers': [{'index': -1, 'values': [-1, -3]},
                                    {'index': -2, 'values': [-4, -2]},
                                    {'index': -3, 'values': [-2, -1]},
                                    {'index': -4, 'values': [-1,2]}]}]
# indexes = [0,1]

In [129]:
# def generate_feature_map_without_word_piece(features):
#     # need to double check and see why this is happening
#     new_features = []
#     i = 0
#     while(i < len(features)):
#         if features[i]['token'] == '[CLS]' or features[i]['token'] == '[SEP]':
#             i += 1
#             continue
#         captured_indexes = []
#         for j in range(i + 1, len(features)):
#             if not features[j]['token'].startswith('##'):
#                 break
#             captured_indexes.append(j)
#         if len(captured_indexes) == 0: 
#             new_features.append(features[i])
#             i += 1
#             continue
#         sum_indexes = [i]
#         sum_indexes.extend(captured_indexes)
#         new_feature = average_over_token_embedding(sum_indexes, features)
#         new_features.append(new_feature)
#         i = captured_indexes[-1] + 1
        
#     # rewrite in the elmo format as well
#     new_features_map = [] # we are converting from the (token, layers) shape to (layers, token) shape
#     layer_minus1 = []; layer_minus2 = []; layer_minus3 = []; layer_minus4 = [];
#     for token in new_features:
#         layer_minus1.append({'token': token['token'], 'features': token['layers'][0]['values']})
#         layer_minus2.append({'token': token['token'], 'features': token['layers'][1]['values']})
#         layer_minus3.append({'token': token['token'], 'features': token['layers'][2]['values']})
#         layer_minus4.append({'token': token['token'], 'features': token['layers'][3]['values']})
#     new_features_map.append({'index': -1, 'values': layer_minus1})
#     new_features_map.append({'index': -2, 'values': layer_minus2})
#     new_features_map.append({'index': -3, 'values': layer_minus3})
#     new_features_map.append({'index': -4, 'values': layer_minus4})
#     return new_features_map
        

In [130]:
generate_feature_map_without_word_piece(features)

[{'index': -1,
  'values': [{'token': '3', 'features': [-1, -3]},
   {'token': 'a##b', 'features': [0.0, -0.5]},
   {'token': '3##b##b', 'features': [-1.0, -3.0]},
   {'token': '3', 'features': [-1, -3]}]},
 {'index': -2,
  'values': [{'token': '3', 'features': [-4, -2]},
   {'token': 'a##b', 'features': [0.5, 1.5]},
   {'token': '3##b##b', 'features': [-2.6666667, -1.3333334]},
   {'token': '3', 'features': [-4, -2]}]},
 {'index': -3,
  'values': [{'token': '3', 'features': [-2, -1]},
   {'token': 'a##b', 'features': [0.5, 1.5]},
   {'token': '3##b##b', 'features': [-2.6666667, -1.6666666]},
   {'token': '3', 'features': [-2, -1]}]},
 {'index': -4,
  'values': [{'token': '3', 'features': [-1, 2]},
   {'token': 'a##b', 'features': [0.5, 0.0]},
   {'token': '3##b##b', 'features': [-1.0, 0.0]},
   {'token': '3', 'features': [-1, 2]}]}]

In [108]:
a = {2: [1,2]}

In [54]:
np.mean([a[2], [3,4]], axis=1)

array([1.5, 3.5])

In [128]:
# def average_over_token_embedding(indexes, features):
#     new_feature = collections.OrderedDict()
#     new_token = ''
#     new_layers = []
    
#     layer_minus_1 = []; layer_minus_2 = []; layer_minus_3 = []; layer_minus_4 = [];
#     for index in indexes:
#         layer_minus_1.append(features[index]['layers'][0]['values'])
#         layer_minus_2.append(features[index]['layers'][1]['values'])
#         layer_minus_3.append(features[index]['layers'][2]['values'])
#         layer_minus_4.append(features[index]['layers'][3]['values'])
#         new_token += features[index]['token']
    
#     new_layers.append({'index': -1, 'values': list(np.mean(layer_minus_1, axis=0, dtype=np.float32))})
#     new_layers.append({'index': -2, 'values': list(np.mean(layer_minus_2, axis=0, dtype=np.float32))})
#     new_layers.append({'index': -3, 'values': list(np.mean(layer_minus_3, axis=0, dtype=np.float32))})
#     new_layers.append({'index': -4, 'values': list(np.mean(layer_minus_4, axis=0, dtype=np.float32))})
    
#     new_feature['token'] = new_token
#     new_feature['layers'] = new_layers
#     return new_feature

In [63]:
average_over_token_embedding(indexes, features)

{'token': 'a##b',
 'layers': [{'index': -1, 'values': [0.0, -0.5]},
  {'index': -2, 'values': [0.5, 1.5]},
  {'index': -3, 'values': [0.5, 1.5]},
  {'index': -4, 'values': [0.5, 0.0]}]}

In [48]:
len(embeddings[0])

4

In [31]:
len(embeddings[0])

4

In [34]:
%load_ext autoreload

In [35]:
%autoreload
import sys
sys.path.append('..')
from relation_extraction.data import utils
import numpy as np

In [37]:
bert_embeddings = utils.get_bert_embeddings('/crimea/geeticka/data/relation_extraction/semeval2010/pre-processed/original/bert/train_original_border_50.json')

JSONDecodeError: Extra data: line 2 column 1 (char 852881)

In [19]:
elmo_embeddings = utils.get_elmo_embeddings('/crimea/geeticka/data/relation_extraction/semeval2010/pre-processed/original/elmo/train_original_border_-1.hdf5')

In [24]:
len(elmo_embeddings[0])

8000

In [25]:
for item in elmo_embeddings[0]:
    print(len(item))
    for i_item in item:
        print(len(i_item))
        for ii_item in i_item:
            print(len(ii_item))
            for iii_item in i_item:
                print(len(iii_item))
                break
            break
        break
    break

3
17
1024
1024
